# Precision recall e ROC

### ROC
TruePositiveRate = TruePositives / (TruePositives + False Negatives)   (RECALL)

FalsePositiveRate = FalsePositives / (FalsePositives + TrueNegatives)

<img src="ROC-Curve.jpg" width=600 height=600 />

### Precision Recall
Precision = TruePositives / (TruePositives + FalsePositives)

Recall = TruePositives / (TruePositives + FalseNegatives)

<img src="Precision-Recall-Curve.jpg" width=600 height=600 />

<img src="Precisionrecall.svg" width=300 height=300 />

https://sinyi-chou.github.io/classification-pr-curve/
#

When data is imbalanced, the AUC might not reflect the true performance of the classifier. The definition of the False Positive Rate (FPR), is the number of false positives divided by the number of negative samples. FPR is considered better when it’s smaller since it indicates fewer false positives. In imbalanced data, the FPR tends to stay at small values due to the large numbers of negatives (i.e. making the denominator large). Thus, FPR becomes less informative for the model performance in this situation.  - 
#### La FPR rimane piccola a causa dello sbilanciamento, non a causa della bont del modello